<a href="https://www.kaggle.com/code/naveenapaleti/worldbank-population-2000-2024?scriptVersionId=264757182" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# World Population from 2000 to 2024

### Import Libraries

In [ ]:
import requests
import pandas as pd

### Parameters

In [ ]:
YEARS = "2000:2024"
BASE = "https://api.worldbank.org/v2"
COUNTRIES = "all" 

INDICATORS = {
    "total_population": "SP.POP.TOTL",
    "male_population": "SP.POP.TOTL.MA.IN",
    "female_population": "SP.POP.TOTL.FE.IN",
    "children_population": "SP.POP.0014.TO",  # ages 0–14
}

### Function to pull indicators

In [ ]:
def wb_pull(indicator_code: str) -> pd.DataFrame:
    """Fetch a World Bank indicator for all countries and years."""
    url = f"{BASE}/country/{COUNTRIES}/indicator/{indicator_code}?date={YEARS}&format=json&per_page=20000"
    resp = requests.get(url, timeout=60)
    resp.raise_for_status()
    meta, data = resp.json()
    
    pages = meta.get("pages", 1)
    rows = data or []
    for p in range(2, pages + 1):
        more = requests.get(url + f"&page={p}", timeout=60).json()[1] or []
        rows.extend(more)

    out = []
    for d in rows:
        if not d or d.get("value") is None:
            continue
        out.append({
            "iso3": d["countryiso3code"],
            "country": d["country"]["value"],
            "year": int(d["date"]),
            indicator_code: float(d["value"])
        })
    return pd.DataFrame(out)


### merge

In [ ]:
dfs = []
for nice_name, code in INDICATORS.items():
    df = wb_pull(code).rename(columns={code: nice_name})
    dfs.append(df)

pop = dfs[0]
for df in dfs[1:]:
    pop = pop.merge(df, on=["iso3", "country", "year"], how="outer")

# Filter by year
pop = pop[(pop["year"] >= 2000) & (pop["year"] <= 2024)].copy()
pop = pop.sort_values(["country", "year"]).reset_index(drop=True)


In [ ]:
pop.head()

In [ ]:
pop.tail()

### Save csv file

In [ ]:
output_file = "population_2000_2024.csv"
pop.to_csv(output_file, index=False)